# Ulimisana vs Genetic Algorithm

In [ ]:
!pip install ushiriki_policy_engine_library

In [ ]:
!pip install geneticalgorithm 

In [ ]:
from ulimisana import testFunctions as tf
from ulimisana import transformFunctions as trf
from ulimisana import agent_positions as ps
from ulimisana import trustworthinessFunction as twf
from ulimisana import ubuntuIncentives as ui
from ulimisana import ulimisana as uoa
from ulimisana.ubuntuIncentives import initialise_ubuntuIncentives
from ulimisana.ubuntuIncentives import ubuntuIncentives
import ulimisana.ulimisana as uoa
import numpy as np
import math
import matplotlib.pyplot as plt
from geneticalgorithm import geneticalgorithm as ga
import gym
import ushiriki_policy_engine_library

### Setting up the Ulimisana experiment.

In [ ]:
time_iter = 2   # Iterations.

lb = 0     # Lower bound for design variables.
ub = 1     # Upper bound for design variables.
dim = 10   # Dimensions of the objective function.



popSize = 200  # Size of the Community.
NoOfFamilies = 5  # Number of families in the community.
fam_aveThreshold = 0.8  # Family average threshold (Average weight or fitness value)
com_aveThreshold = 0.8  # Community average threshold (Average weight or fitness value)
ageAverage = 35  # Average age in the community.
ageStdev = 7  # Age standard deviation.


phi = 0.7  # Weighted average on payoffs.


epsilon = 0.15  # Advisor's maximum acceptable level of error.
r = 0.7        # Reputation. 
trustThreshold = 0.45 # Threshold on trust.

objFunction   = tf.MalariaCost # Malaria simulation objective function.
sigFun_term   = trf.tanh # Transformation function.
sigFun_weight = trf.tanh # Transformation function.

'''
 x_info    : Gives information about each agent(individual). Their age, position at the end of all iterations, 
             their objective value (payoff) and family they belong to.
 x_pos     : Gives the changes in position over all the iterations.
 ind_val   : Gives the individual's payoffs for each iteration.
 fam_val   : Gives family payoffs for each iteration.
 com_val   : Gives community payoffs for each iteration.
 adv_trust : Gives the changes of trustworthiness each family had towards their advisors. 
 trust     : Gives the changes of trustworthiness each family had towards all other families (This is computed using advisor's trustworthiness)

'''
# Evaluating the objective function over 'n' iterations.
list_of_best=[]
for i in range(5):
    x_info,x_pos,ind_val,fam_val,comm_val,adv_trust,trust_matrix = uoa.ulimisana(time_iter,popSize,NoOfFamilies,ageAverage,ageStdev,
              objFunction,dim,lb,ub,sigFun_term,sigFun_weight,fam_aveThreshold,com_aveThreshold,phi,epsilon,r,
              trustThreshold)
    best_ind_payoffs=max(np.array(ind_val)[0])
    list_of_best.append(best_ind_payoffs)
    best_ind_payoffs=0
print(list_of_best)

### Setting up the GA experiment

In [ ]:
# Creating the Malaria platform (environment)
iterations = 0
environment = gym.make("ChallengePolicy-v0", 
                     userID="61122946-1832-11ea-8d71-362b9e155667",
                     baseuri="https://reward-service.9xxws07e6gx.us-south.codeengine.appdomain.cloud")
environment.reset()

# Defining the Malaria environment as an objective function to be evaluated by Genetic algorithm.
def malariaCost(x):
    global iterations
    global environment
    if iterations == 0:
        print('Creating environment')
        environment = gym.make("ChallengePolicy-v0", 
                     userID="61122946-1832-11ea-8d71-362b9e155667",
                     baseuri="https://reward-service.9xxws07e6gx.us-south.codeengine.appdomain.cloud")
        environment.reset()
    
    policy = {}
    j = 0
    for i in range(1, 6):
            
                                                 
        itn = x[j]
        irs = x[j+1]
        j += 2
        policy[i] = (itn, irs)
    print(f'{2000-iterations} left!')
    iterations += 5
    
    if iterations >= 2000:
        iterations = 0
        print(f'Iterations reset to {iterations}')
    
    return -1*environment.step(policy)[1][-1]



varbound=np.array([[0,1]]*10)

algorithm_param = {'max_num_iteration': 3,\
                   'population_size':130,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}

# Evaluating the objective function over 'n' iterations.
best_collection = []
for i in range(5):
    model=ga(function=malariaCost,dimension=10,\
         variable_type='real',\
             variable_boundaries=varbound,
             algorithm_parameters=algorithm_param)
    all_model_results = model.run()
    best_solution = np.mean(model.report)
    updated_list_of_best = best_collection.append(abs(best_solution))
    best_solution = 0
print(best_collection)    

### Generating plots for Ulimisana vs GA

In [ ]:
np.random.seed(10)
data1 = list_of_best
data2 = best_collection
data = [data1, data2]
fig = plt.figure(figsize = (6, 4))
plt.boxplot(data)


plt.xticks([1, 2], ["Ulimisana", "GA"], rotation=10)
#plt.xlabel('Ulimisana')
plt.ylabel('Episodic Payoffs')
plt.axis([None, None, 0, 400])
plt.title('Best Ulimisana Ubuntu Incentive Parameters vs Genetic Algorithm')
plt.savefig('Best_Ub_vs_GA')
plt.show()